# Projeto Banco de Dados I
### Professor Rogério Mainardes
### Alunos Erick Henrique e Paulo Vitor 

Esse projeto consiste na análise descritiva e de 10 perguntas que serão respondidas através do uso de SQL para um dataset.
O dataset escolhido está disponível no [github](https://github.com/ValdisW/datasets/blob/master/video-game-sales.csv).

Os dados escolhidos mostram as vendas de jogos de videogame lançados entre 1980 e 2020 que tiveram mais de 100.000 vendas nas regiões América do Norte, União Europeia, Japão ou outros (resto do mundo). Na tabela, as colunas *_sales mostram as vendas em milhões de unidades.

Análise descritiva:

- Qual as tipagens das colunas?;
- Quantos nulos temos em cada coluna?;
- Quantos valores únicos temos nas colunas categoricas?;
- Existem outliers nos dados numéricos? Se sim, como lidar com eles?;

10 perguntas:

- Qual a soma de todos os jogos vendidos no Wii?
- Quais foram os jogos mais vendidos entre 2010 e 2016?
- Qual região foi responsável pela maior quantidade de vendas  de jogos?
- Quais as 3 plataformas com mais jogos vendidos?
- Quais os 10 jogos mais vendidos no japão?
- Quantos jogos foram lançados no ano de 2008?
- Quais as 3 plataformas com mais jogos lançados?
- A soma das vendas dos 3 jogo lançado em maior número de plataforma?

In [1]:
-- Criação da tabela videogame_sales no postgres
CREATE TABLE videogame_sales(
    ranking integer,
    game_name varchar(200),
    platform varchar(20),
    release_year smallint,
    genre varchar(20),
    publisher varchar(50),
    na_sales real,
    eu_sales real,
    jp_sales real,
    other_sales real,
    global_sales real
);

In [ ]:
-- cópia dos dados do arquivo videogame-sales para o postgres
COPY videogame_sales 
FROM 'caminho/do/arquivo/arquivo.csv'
WITH(
    FORMAT csv,
    DELIMITER ',', 
    HEADER,
    NULL 'N/A'
);

### Conectando com o banco de dados

In [4]:
# Instalando libs necessárias
!pip install sqlalchemy
!pip install pandas
!pip install pandasql

^C
ERROR: Operation cancelled by user


In [31]:
# Importa as libs instaladas

from sqlalchemy import create_engine
import pandas as pd
import pandasql as ps

In [87]:
# Configura conexão com o banco

SGBD = "postgresql+psycopg2"
USER = "postgres"
SENHA = "123"
HOST = "localhost"
DATABASE = "Video game dataset"

engine = create_engine(f"{SGBD}://{USER}:{SENHA}@{HOST}/{DATABASE}")
conn = engine.connect()

In [52]:
# Consulta teste
pd.read_sql_query(
    """
    SELECT 
        * 
    FROM 
        videogame_sales
    LIMIT 4
    """, 
    con=conn
)

,ranking,game_name,platform,release_year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00


### Análise Descritiva

#### Tipagem das colunas

In [62]:
pd.read_sql_query(
    """
    SELECT 
        column_name, data_type
    FROM 
        information_schema.columns
    WHERE
        table_schema = 'public' AND table_name = 'videogame_sales'
    """, 
    con=conn
)

,column_name,data_type
0,ranking,integer
1,game_name,character varying
2,platform,character varying
3,release_year,smallint
4,genre,character varying
5,publisher,character varying
6,na_sales,real
7,eu_sales,real
8,jp_sales,real
9,other_sales,real


Information schema é um schema criado automaticamente pelo postgres e que contém informações de todas as colunas no banco de dados. Filtramos esse schema para mostrar apenas dados da tabela que criamos (videogame_sales) e, especificamente, as colunas column_name e data_type.
Para verificar outras informações presentes nesse schema, altere o 'SELECT' para '*'.

#### Quantidade de valores nulos em cada coluna

In [71]:
pd.read_sql_query(
    """
    SELECT
	    COUNT(*) FILTER (WHERE ranking IS NULL) AS nulos_ranking,
	    COUNT(*) FILTER (WHERE game_name IS NULL) AS nulos_game_name,
        COUNT(*) FILTER (WHERE platform IS NULL) AS nulos_platform,
        COUNT(*) FILTER (WHERE release_year IS NULL) AS nulos_release_year,
        COUNT(*) FILTER (WHERE genre IS NULL) AS nulos_genre,
        COUNT(*) FILTER (WHERE publisher IS NULL) AS nulos_publisher,
        COUNT(*) FILTER (WHERE na_sales IS NULL) AS nulos_na_sales,
        COUNT(*) FILTER (WHERE eu_sales IS NULL) AS nulos_eu_sales,
        COUNT(*) FILTER (WHERE jp_sales IS NULL) AS nulos_jp_sales,
        COUNT(*) FILTER (WHERE other_sales IS NULL) AS nulos_other_sales,
        COUNT(*) FILTER (WHERE global_sales IS NULL) AS nulos_global_sales
    FROM
        videogame_sales
    """, 
    con=conn
)

,nulos_ranking,nulos_game_name,nulos_platform,nulos_release_year,nulos_genre,nulos_publisher,nulos_na_sales,nulos_eu_sales,nulos_jp_sales,nulos_other_sales,nulos_global_sales
0,0,0,0,271,0,58,0,0,0,0,0


A consulta acima conta todos os elementos presentes em uma coluna em que foi aplicado um filtro que mostra apenas os elementos de valor NULL, repetindo essa mesma contagem e aplicação de filtro para cada uma das colunas.
Assim, temos a contagem de elementos nulos em cada uma das colunas.

#### Valores únicos nas colunas de variáveis categóricas

Temos as seguintes colunas como representantes de variáveis categóricas:
- Plataforma
- Ano de lançamento
- Gênero
- Empresa desenvolvedora

A consulta abaixo pode nos mostrar o número de categorias distintas (valores únicos) em cada uma das colunas

In [91]:
pd.read_sql_query(
    """
    SELECT 
	    count(DISTINCT platform) AS unique_platform_count,
	    count(DISTINCT release_year) AS unique_release_year_count,
	    count(DISTINCT genre) AS unique_genre_count,
	    count(DISTINCT publisher) AS unique_publisher_count
    FROM
        videogame_sales
    """, 
    con=conn
)

,unique_platform_count,unique_release_year_count,unique_genre_count,unique_publisher_count
0,31,39,12,578


A consulta acima conta os valores distintos em cada uma das colunas e nos retorna essa contagem.

#### Outliers nas colunas numéricas

Como reperesentantes de variáveis numéricas temos as seguintes colunas:
- na_sales
- eu_sales
- jp_sales
- other_sales
- global_sales

##### Definição de outlier

Definiremos como outlier um valor que estiver acima de Q3 + 1.5 * IQR ou abaixo de Q1 - 1.5 * IQR.<br>
Q1 e Q3 são os valores do primeiro e terceiro quartis.<br>
IQR (Intervalo Interquartil) é a diferença Q3 - Q1. 

##### Outliers em na_sales

In [75]:
pd.read_sql_query(
    """
    WITH stats AS (
      SELECT
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY na_sales) AS q1,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY na_sales) AS q3
      FROM
        videogame_sales
    )
    SELECT
      COUNT(na_sales)
    FROM
      videogame_sales, stats
    WHERE
      na_sales < stats.q1 - 1.5 * (stats.q3 - stats.q1) OR
      na_sales > stats.q3 + 1.5 * (stats.q3 - stats.q1);
    """, 
    con=conn
)

,count
0,1711


Temos Q1 = 0 e Q3 = 0.24.<br>
Assim, nosso IQR vale 0.36.<br>
São outliers valores acima de 0.6 ou abaixo de -0.36.<br>
Como não temos valores negativos para vendas, não há outliers "para baixo".<br>
Há 1711 outliers "para cima".<br> 
Não há motivo para desconsiderá-los, uma vez que não representam erros de medida ou algo do tipo. São apenas jogos que tiveram vendas excepcionais quando comparados ao resto.

##### Outliers em eu_sales

In [82]:
pd.read_sql_query(
    """
    WITH stats AS (
      SELECT
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY eu_sales) AS q1,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY eu_sales) AS q3
      FROM
        videogame_sales
    )
    SELECT
      COUNT(eu_sales)
    FROM
      videogame_sales, stats
    WHERE
      eu_sales < stats.q1 - 1.5 * (stats.q3 - stats.q1) OR
      eu_sales > stats.q3 + 1.5 * (stats.q3 - stats.q1)
    """, 
    con=conn
)

,count
0,2081


Temos Q1 = 0 e Q3 = 0.11.<br>
Assim, nosso IQR é 0.11.<br>
São outliers valores abaixo de -0.165 ou acima de 0.275.<br>
Da mesma forma que em na_sales, não temos outliers "para baixo".<br>
Existem 2081 outliers "para cima".<br>
O tratamento dado aos outliers será o mesmo de na_sales.<br>

##### Outliers em jp_sales

In [90]:
pd.read_sql_query(
    """
    WITH stats AS (
      SELECT
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY jp_sales) AS q1,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY jp_sales) AS q3
      FROM
        videogame_sales
    )
    SELECT
      COUNT(jp_sales)
    FROM
      videogame_sales, stats
    WHERE
      jp_sales < stats.q1 - 1.5 * (stats.q3 - stats.q1) OR
      jp_sales > stats.q3 + 1.5 * (stats.q3 - stats.q1)
    """, 
    con=conn
)

,count
0,2577


Temos Q1 = 0 e Q3 = 0.04.<br>
Assim, nosso IQR é 0.04.<br>
Serão outliers valores abaixo de -0.02 ou acima de 0.1.<br>
Não temos outliers "para baixo".<br>
Existem 2577 outliers "para cima".<br>
O tratamento dado aos outliers será o mesmo de antes.<br>

##### Outliers em other_sales

In [94]:
pd.read_sql_query(
    """
    WITH stats AS (
      SELECT
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY other_sales) AS q1,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY other_sales) AS q3
      FROM
        videogame_sales
    )
    SELECT
      COUNT(other_sales)
    FROM
      videogame_sales, stats
    WHERE
      other_sales < stats.q1 - 1.5 * (stats.q3 - stats.q1) OR
      other_sales > stats.q3 + 1.5 * (stats.q3 - stats.q1)
    """, 
    con=conn
)

,other_sales
0,0.10
1,0.10
2,0.10
3,0.10
4,0.10
...,...
1827,3.31
1828,4.14
1829,7.53
1830,8.46


Temos Q1 = 0 e Q3 = 0.04.<br>
Assim, nosso IQR é 0.04.<br>
Serão outliers valores abaixo de -0.02 ou acima de 0.1.<br>
Não temos outliers "para baixo".<br>
Existem 1832 outliers "para cima".<br>
O tratamento dado aos outliers será o mesmo de antes.<br>

##### Outliers em global_sales

In [96]:
pd.read_sql_query(
    """
    WITH stats AS (
      SELECT
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY global_sales) AS q1,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY global_sales) AS q3
      FROM
        videogame_sales
    )
    SELECT
      COUNT(global_sales)
    FROM
      videogame_sales, stats
    WHERE
      global_sales < stats.q1 - 1.5 * (stats.q3 - stats.q1) OR
      global_sales > stats.q3 + 1.5 * (stats.q3 - stats.q1)
    """, 
    con=conn
)

,count
0,1893


Temos Q1 = 0.06 e Q3 = 0.47.<br>
Assim, nosso IQR é 0.39.<br>
Serão outliers valores abaixo de -0.535 ou acima de 1.055.<br>
Não temos outliers "para baixo".<br>
Existem 1893 outliers "para cima".<br>
O tratamento dado aos outliers será o mesmo de antes.<br>

#### 10 Perguntas

##### Questão 1 - Qual a soma de todos os jogos vendidos no Wii?

In [ ]:
pd.read_sql_query(
    """
        SELECT 
            SUM(global_sales)
        FROM 
            videogame_sales
        WHERE 
            platform = 'Wii'
    """, 
    con=conn
)

Essa query filtra a plataforma com nome "Wii" e soma todos os seus global_sales.

##### Questão 2 - Quais foram 5 os jogos mais vendidos entre 2010 e 2016?

In [ ]:
pd.read_sql_query(
    """
        SELECT 
            game_name, 
            SUM(global_sales) AS global_sales_sum
        FROM 
            videogame_sales
        WHERE 
            release_year <= 2016 AND release_year >= 2010
        GROUP BY 
            game_name
        ORDER BY 
            global_sales_sum DESC
        LIMIT 
            5
    """, 
    con=conn
)

Essa query agrupa todos os jogos com mesmo nome, filtra os jogos entre 2010 e 2016 e soma seus global_sales. Ao final, ordena os jogos por venda e limita a quantidade de jogos mostrados em 5.

##### Questão 3 - Qual região foi responsável pela maior quantidade de vendas  de jogos?

In [ ]:
pd.read_sql_query(
    """
        SELECT 
            'na_sales' AS region_sales, 
            SUM(na_sales) AS valor
        FROM 
            videogame_sales
        UNION ALL
        SELECT 
            'eu_sales' AS region_sales, 
            SUM(eu_sales) AS valor
        FROM 
            videogame_sales
        UNION ALL
        SELECT 
            'jp_sales' AS region_sales, 
            SUM(jp_sales) AS valor
        FROM 
            videogame_sales
        UNION ALL
        SELECT 
            'other_sales' AS region_sales, 
            SUM(other_sales) AS valor
        FROM 
            videogame_sales
        ORDER BY 
            valor DESC
        LIMIT 
            1
    """, 
    con=conn
)

Essa query soma as vendas de um região e cria uma string para nomeá-la, em seguida, faz o mesmo com as outras e as une em uma única tabela. Por fim, ordena os valores de venda em ordem decrescente e mostra apenas o primeiro dado.

#### Questão 4 - Quais as 3 plataformas com mais jogos vendidos?

In [ ]:
pd.read_sql_query(
    """
        SELECT 
            platform, 
            SUM(global_sales) AS sum_global_sales
        FROM 
            videogame_sales
        GROUP BY 
            platform
        ORDER BY 
            sum_global_sales DESC
        LIMIT 
            3
    """, 
    con=conn
)

Essa query agrupa as plataformas de mesmo nome e soma os seus global_sales. Em seguida, ela ordena em ordem decrescente pelo sum_global_sales e limita a visualização em 3.

##### Questão 5 - Quais os 10 jogos mais vendidos no japão?

In [ ]:
pd.read_sql_query(
    """
        SELECT 
            game_name, 
            SUM(jp_sales) AS sum_jp_sales
        FROM 
            videogame_sales
        GROUP BY 
            game_name
        ORDER BY 
            sum_jp_sales DESC
        LIMIT 
            10
    """, 
    con=conn
)

Essa query junta os jogos de mesmo nome e soma as suas jp_sales. Em seguida ordena em ordem decrescente pelo sum_jp_sales e limita sua visualização em 10.

##### Questão 6 - Quantos jogos foram lançados no ano de 2008?

In [ ]:
pd.read_sql_query(
    """
        SELECT 
            release_year, 
            COUNT(game_name) AS count_game_name
        FROM 
            videogame_sales
        WHERE 
            release_year = 2008
        GROUP BY 
            release_year
    """, 
    con=conn
)

Essa query filtra todos os jogos que foram lançados em 2008 e conta sua quantidade.

##### Questão 7 - Quais as 3 plataformas com mais jogos lançados?

In [ ]:
pd.read_sql_query(
    """
        SELECT 
            platform, 
            COUNT(game_name) AS count_game_name
        FROM 
            videogame_sales
        GROUP BY 
            platform
        ORDER BY 
            count_game_name DESC
        LIMIT 
            3
    """, 
    con=conn
)

Essa query agrupa as plataformas de mesmo nome e conta a quantidade de jogos em cada uma. Em seguida ordena de forma decrescente pela quantidade de jogos e limita sua visualização em 3.

##### Questão 8 - A soma das vendas dos 3 jogos lançados em maior número de plataforma?

In [ ]:
pd.read_sql_query(
    """
        SELECT 
            game_name, 
            COUNT(platform) AS count_platform, 
            SUM(global_sales) AS sum_global_sales
        FROM 
            videogame_sales
        GROUP BY 
            game_name
        ORDER BY 
            count_platform DESC
        LIMIT 
            3
    """, 
    con=conn
)

Essa query agrupa os jogos de mesmo nome, conta a quantidade de plataformas de cada jogo e soma seu global_sales. Em seguida, ordena em ordem decrescente pelo count_platform e limita sua visualização em 3.

##### Questão 9 - Quantos jogos de cada gênero cada desenvolvedora lançou?

In [ ]:
pd.read_sql_query(
    """
        SELECT 
            publisher,
            genre,
            COUNT(genre) AS count_genre
        FROM
            videogame_sales
        GROUP BY 
            genre, 
            publisher
        ORDER BY 
            publisher, 
            count_genre DESC
    """, 
    con=conn
)

Essa query agrupa os gêneros e desenvolvedoras e mostra em ordem alfabética de desenvolvedores e decrescente de gênero a quantidade de jogos de um mesmo gênero lançado por uma desenvolvedora.